In [1]:
import pandas as pd
import json
from pprint import pprint

from urllib import parse
import simplejson, urllib
import urllib.request

In [2]:
search_keyword =""

def req_get_list(search_keyword):
    target_url = parse.urlparse(f"https://www.wanted.co.kr/api/v4/search/summary?&locations=all&job_sort=company.response_rate_order&years=-1&country=kr&query={search_keyword}")
    query = parse.parse_qs(target_url.query)
    url_query = parse.urlencode(query, doseq=True)
    result_url ='https://www.wanted.co.kr/api/v4/search/summary?&'+ url_query

    r = simplejson.load(urllib.request.urlopen(result_url))
    return r

In [3]:
def binary_search_id_list(value,id_list):
    sorted_id_list  = id_list.sort()
    start = 0
    last = len(sorted_id_list)
    while(start<=last):
        if(id_list[last//2] ==value):
            return int(0)
        if(id_list[last//2]>value):
            last=id_list[last//2]-1
        else:
            start = id_list[last//2]+1
    return int(1)


In [8]:
def deduplicatesId_N_makeDf(key_word_list):
    # 동적변수 할당해서 검색할 데이터를 나누어주고 이렇게 불러와준 id를 모아놓고 return 
    id_list=[]
    company_name_list =[]
    company_industry_list  =[]
    position_list =[]
    opt=1

    for i,v in enumerate(key_word_list):
        locals()[f"r{str(i+1)}"]= req_get_list(v)
        if(len(id_list)>=0):
            opt = binary_search_id_list(locals()[f"r{str(i+1)}"]['jobs']['data']['id'],id_list)

        if(opt==1):
            id_list.append(locals()[f"r{str(i+1)}"]['jobs']['data']['id'])    
            company_name_list.append(locals()[f"r{str(i+1)}"]['jobs']['data']['company']['name'])
            company_industry_list.append(locals()[f"r{str(i+1)}"]['jobs']['data']['company']['industry_name'])
            position_list.append(locals()[f"r{str(i+1)}"]['jobs']['data']['position'])
        else:
            continue
    
    df = pd.DataFrame()
    df['id'] = id_list 
    df['company_name'] = company_name_list
    df['company_industry_name'] = company_industry_list
    df['position'] = position_list

    
    
    return df,id_list

In [5]:
def req_get_jobcard(card_num,df):
    job_card_url ='https://www.wanted.co.kr/api/v4/jobs/'+str(card_num)

    r = simplejson.load(urllib.request.urlopen(job_card_url))

    benefits = r['job']['detail']['benefits']
    intro = r['job']['detail']['intro']
    main_tasks = r['job']['detail']['main_tasks']
    preferred_points = r['job']['detail']['preferred_points']
    requirements = r['job']['detail']['requirements']

    # row 데이터 저장
    df.loc[df.shape[0]-1] = [requirements,preferred_points,main_tasks,intro,benefits]
    
    return df

In [9]:
card_num = ''

keyword =["데이터 엔지니어",'데이터엔지니어','data engineer','MLOps']

company_df,search_list = deduplicatesId_N_makeDf(keyword)



job_detail = pd.DataFrame(columns=['requirements','preferred_points','main_tasks','intro','benefits'])
for num in search_list:
    job_detail = req_get_jobcard(num,job_detail)


print(company_df)
print(job_detail)


TypeError: list indices must be integers or slices, not str

In [7]:
search_list

[]